# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [1]:
import pandas as pd
import sqlite3
from IPython.display import display

conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",)
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: 
BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, PNF_DATA, ALL_DATA, 


# REQUESTING USER'S ZIP TO COMPUTE PDP_REGION_CODE

In [2]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


# REQUESTING DRUGS FROM USER, YIELDING RXCUIs

In [4]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):None\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql("select * from RXCUI_DATA where upper(PSN) like '"+drug+"%' and strength like '"+strength+" %MG' and GENERIC_RXCUI is ''",conn).drop_duplicates(subset='SXDG_RXCUI',keep='last')['RXCUI'].values[0]
print(user_generics)

Enter how many drugs you wish to search: 3
Enter a generic drug name: metoprolol succinate
Enter a desired strength: 50
Enter a generic drug name: metformin
Enter a desired strength: 500
Enter a generic drug name: allopurinol
Enter a desired strength: 300
{('METOPROLOL SUCCINATE', '50'): '866436', ('METFORMIN', '500'): '861007', ('ALLOPURINOL', '300'): '197320'}


##### GETTING DATA FOR FORMULARY_IDs IN REGION

In [5]:
out = pd.read_sql("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE \
            from \
                FORMULARY_DATA join PLAN_INFO \
                on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%'\
            order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)
display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0


# YIELDING PLAN SELECTION OUTPUT TABLE
###### EXTREMELY FAST NOW

1. CONSTRUCTING INITIAL TABLE
2. PUTTING EACH DRUG IN THE TABLE, COMPUTING COST
3. SHOWING TOTAL COST FOR EACH ITERATION

In [6]:
out = pd.read_sql("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE \
            from \
                FORMULARY_DATA join PLAN_INFO \
                on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%'\
            order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True)

for key in user_generics:
    out = pd.concat([out,
               pd.read_sql(" ".join(("select \
                        PLAN_INFO.CONTRACT_NAME, \
                        PLAN_INFO.PLAN_NAME, \
                        PLAN_INFO.CONTRACT_ID, \
                        PLAN_INFO.PLAN_ID, \
                        PLAN_INFO.SEGMENT_ID, \
                        PLAN_INFO.FORMULARY_ID, \
                        PLAN_INFO.PREMIUM, \
                        PLAN_INFO.DEDUCTIBLE, \
                        FORMULARY_DATA.RXCUI as "+key[0].replace(" ","_")+"_RXCUI, \
                        FORMULARY_DATA.PRIOR_AUTHORIZATION_YN as "+key[0].replace(" ","_")+"_PA, \
                        FORMULARY_DATA.TIER_LEVEL_VALUE as "+key[0].replace(" ","_")+"_TIER, \
                        BENEFICIARY_COST.COST_AMT_PREF as "+key[0].replace(" ","_")+"_CAP \
                from \
                    FORMULARY_DATA join PLAN_INFO \
                        on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
                    join BENEFICIARY_COST \
                        on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                           PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                           cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                    cast(BENEFICIARY_COST.TIER as integer) \
                where \
                      PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                      FORMULARY_DATA.RXCUI is "+user_generics[key]+" and \
                      PLAN_INFO.CONTRACT_ID like 'S%' and \
                      (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
                      cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
                      BENEFICIARY_COST.DAYS_SUPPLY = 1 and \
                      cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1\
                order by PLAN_INFO.FORMULARY_ID").split()),conn)\
    .drop_duplicates(subset='FORMULARY_ID',keep='first')\
    .reset_index(drop=True)\
    [[key[0].replace(" ","_")+'_RXCUI',
      key[0].replace(" ","_")+'_PA',
      key[0].replace(" ","_")+'_TIER',
      key[0].replace(" ","_")+'_CAP']]],axis=1)
    #ADD IN COST COLUMN
    if 'ANNUAL_TOTAL_COST' in out.columns:
        del out['ANNUAL_TOTAL_COST']
    if 'MONTHLY_COPAY' in out.columns:
        del out['MONTHLY_COPAY']
    out['ANNUAL_TOTAL_COST'] = (out[[c for c in out.columns if c[-3:] == 'CAP']]*12).sum(axis=1) + out['PREMIUM']*12 + out['DEDUCTIBLE']
    out['MONTHLY_COPAY'] = out[[c for c in out.columns if c[-3:] == 'CAP']].sum(axis=1)
    out = out.sort_values(by='ANNUAL_TOTAL_COST').reset_index(drop=True)
    display(out)



,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,579.6,14.0
1,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,866436,N,2,3.0,677.4,3.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,682.8,4.0
3,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,2,4.0,697.8,4.0
4,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,698.4,2.0
5,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,220,0,18152,22.6,405.0,866436,N,2,4.0,724.2,4.0
6,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,1,0.0,726.6,0.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,767.8,2.0
8,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,1,1.0,781.8,1.0
9,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0,866436,N,1,1.0,815.4,1.0


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,METFORMIN_RXCUI,METFORMIN_PA,METFORMIN_TIER,METFORMIN_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,861007,N,1,6.0,651.6,20.0
1,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,866436,N,2,3.0,861007,N,1,1.0,689.4,4.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,861007,N,1,1.0,694.8,5.0
3,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,2,4.0,861007,N,1,0.0,697.8,4.0
4,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,861007,N,1,1.0,710.4,3.0
5,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,220,0,18152,22.6,405.0,866436,N,2,4.0,861007,N,1,0.0,724.2,4.0
6,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,1,0.0,861007,N,1,1.0,738.6,1.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,861007,N,1,1.0,779.8,3.0
8,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,1,1.0,861007,N,1,4.0,829.8,5.0
9,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Value (PDP),S5660,106,0,18154,34.0,405.0,866436,N,2,3.0,861007,N,1,1.0,861.0,4.0


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,...,METFORMIN_RXCUI,METFORMIN_PA,METFORMIN_TIER,METFORMIN_CAP,ALLOPURINOL_RXCUI,ALLOPURINOL_PA,ALLOPURINOL_TIER,ALLOPURINOL_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,...,861007,N,1,0.0,197320,N,1,0.0,697.8,4.0
1,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,701.4,5.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,706.8,6.0
3,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,722.4,4.0
4,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,...,861007,N,1,6.0,197320,N,1,6.0,723.6,26.0
5,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,220,0,18152,22.6,405.0,866436,N,...,861007,N,1,0.0,197320,N,1,0.0,724.2,4.0
6,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,750.6,2.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,791.8,4.0
8,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,...,861007,N,1,4.0,197320,N,1,4.0,877.8,9.0
9,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare Classic (PDP),S4802,78,0,18443,36.9,405.0,866436,N,...,861007,N,1,1.0,197320,N,1,1.0,883.8,3.0


# FULL FINAL OUTPUT, SORTED BY ANNUAL TOTAL COST

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 22):
    display(out)

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,METOPROLOL_SUCCINATE_RXCUI,METOPROLOL_SUCCINATE_PA,METOPROLOL_SUCCINATE_TIER,METOPROLOL_SUCCINATE_CAP,METFORMIN_RXCUI,METFORMIN_PA,METFORMIN_TIER,METFORMIN_CAP,ALLOPURINOL_RXCUI,ALLOPURINOL_PA,ALLOPURINOL_TIER,ALLOPURINOL_CAP,ANNUAL_TOTAL_COST,MONTHLY_COPAY
0,HUMANA INSURANCE COMPANY,Humana Walmart Rx Plan (PDP),S5884,150,0,18255,20.4,405.0,866436,N,2,4.0,861007,N,1,0.0,197320,N,1,0.0,697.8,4.0
1,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,866436,N,2,3.0,861007,N,1,1.0,197320,N,1,1.0,701.4,5.0
2,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,4.0,861007,N,1,1.0,197320,N,1,1.0,706.8,6.0
3,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,2.0,861007,N,1,1.0,197320,N,1,1.0,722.4,4.0
4,SILVERSCRIPT INSURANCE COMPANY,SilverScript Choice (PDP),S5601,8,0,18419,34.3,0.0,866436,N,2,14.0,861007,N,1,6.0,197320,N,1,6.0,723.6,26.0
5,MEDCO CONTAINMENT LIFE AND MEDCO CONTAINMENT NY,Express Scripts Medicare - Saver (PDP),S5660,220,0,18152,22.6,405.0,866436,N,2,4.0,861007,N,1,0.0,197320,N,1,0.0,724.2,4.0
6,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,1,0.0,861007,N,1,1.0,197320,N,1,1.0,750.6,2.0
7,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,2.0,861007,N,1,1.0,197320,N,1,1.0,791.8,4.0
8,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,1,1.0,861007,N,1,4.0,197320,N,1,4.0,877.8,9.0
9,"WELLCARE PRESCRIPTION INSURANCE, INC.",WellCare Classic (PDP),S4802,78,0,18443,36.9,405.0,866436,N,2,1.0,861007,N,1,1.0,197320,N,1,1.0,883.8,3.0


# FINDING LOCATIONS IN ZIP THAT OFFER CHEAPEST PLAN
##### TAKES A LONGER TIME BECAUSE PNF_DATA IS HUGE

In [12]:
for i in range(3):
    display(pd.read_sql("select NCPDP.PHARMACY_NAME, NCPDP.PHONE, PNF_DATA.* \
                 from NCPDP, PNF_DATA\
                 where PNF_DATA.CONTRACT_ID='"+out.iloc[i].CONTRACT_ID+"' and \
                       cast(PNF_DATA.PLAN_ID as integer)="+str(out.iloc[i].PLAN_ID)+" and \
                       cast(PNF_DATA.SEGMENT_ID as integer)="+str(out.iloc[i].SEGMENT_ID)+" and \
                       PNF_DATA.PREFERRED_STATUS_RETAIL='Y' and \
                       PNF_DATA.PHARMACY_ZIPCODE like '"+user_zip[:-2]+"%' and \
                       NCPDP.NPI is substr(PNF_DATA.PHARMACY_NUMBER,3)",conn))

,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,SAMS PHARMACY,2019740556,S5884,150,000,101518984657,07094,Y,N,Y,N,1,1,0,1,1,0,1
1,SAMS PHARMACY,9085879820,S5884,150,000,101104007772,07036,Y,N,Y,N,1,1,0,1,1,0,1


,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,CVS PHARMACY # 08924,2013132254,S5810,278,000,101891015475,07020,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
1,TARGET PHARMACY,9086229003,S5810,278,000,101538320882,07088,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
2,CVS PHARMACY,9083514940,S5810,278,000,101447442843,07083,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
3,CVS PHARMACY,2014203286,S5810,278,000,101801919907,07030,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
4,TARGET PHARMACY,9084442024,S5810,278,000,101972525590,07080,Y,N,Y,N,1,0.4,0.4,0.4,0.4,0.4,0.4
5,SHOPRITE PHARMACY,9734717499,S5810,278,000,101154436939,07055,Y,N,Y,N,1,1,1,1,1,1,1
6,ACME PHARMACY #1893,2018670750,S5810,278,000,101497122246,07086,Y,N,Y,N,1,1,1,1,1,1,1
7,ACME PHARMACY #1096,9082415421,S5810,278,000,101396112231,07033,Y,N,Y,N,1,1,1,1,1,1,1
8,ACME PHARMACY #1097,9087911600,S5810,278,000,101144697962,07080,Y,N,Y,N,1,1,1,1,1,1,1
9,ACME PHARMACY #1782,7329433304,S5810,278,000,101790152536,07066,Y,N,Y,N,1,1,1,1,1,1,1


,PHARMACY_NAME,PHONE,CONTRACT_ID,PLAN_ID,SEGMENT_ID,PHARMACY_NUMBER,PHARMACY_ZIPCODE,PREFERRED_STATUS_RETAIL,PREFERRED_STATUS_MAIL,PHARMACY_RETAIL,PHARMACY_MAIL,IN_AREA_FLAG,BRAND_DISPENSING_FEE_30,BRAND_DISPENSING_FEE_60,BRAND_DISPENSING_FEE_90,GENERIC_DISPENSING_FEE_30,GENERIC_DISPENSING_FEE_60,GENERIC_DISPENSING_FEE_90
0,MEDICINE SHOPPE,9732393339,S5617,249,000,101679685671,07009,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
1,TERRY DRUGS,9733387300,S5617,249,000,101720143936,07003,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
2,RAPPS PHARMACY INC,9087560008,S5617,249,000,101659478576,07060,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
3,WEST END PHARMACY,2014382164,S5617,249,000,101073667424,07070,Y,N,Y,N,1,1,0.5,0.5,0.5,0.5,0.5
4,SHOPRITE PHARMACY,9736748822,S5617,249,000,101871608646,07018,Y,N,Y,N,1,0.4,0,0,0.4,0,0
5,TERRYS FAMILY PHARMACY INC,9732392200,S5617,249,000,101053405464,07044,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
6,HINES PHARMACY,2013482667,S5617,249,000,101770598385,07087,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
7,BELLS PHARMACY,9082760062,S5617,249,000,101639285042,07016,Y,N,Y,N,1,0.5,0.5,0.5,0.6,0.6,0.6
8,HUDACKOS PHARMACY INC,2014364488,S5617,249,000,101316043573,07002,Y,N,Y,N,1,0.75,0.75,0.75,0.75,0.4,0.4
9,SHOPRITE PHARMACY,2019913568,S5617,249,000,101780799551,07031,Y,N,Y,N,1,0.4,0,0,0.4,0,0
